In [6]:
!pip install konlpy

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from collections import Counter

In [247]:
doc = """
"원자력안전위원장 강창순] \"예 알겠습니다.\"\n유성엽 위원] \"이상입니다.\"\n위원장 신학용] \"유성엽 위원님 수고하셨습니다.   마지막으로 유기홍 간사님 질의해 주세요.\"\n유기홍 위원] \"제가 제 질의 전에 간단하게 의사진행발언 하고 하도록 하겠습니다.   사실은 다른 위원님들은 어떻게 생각하실지 모르겠지만 저는 오늘 제대로 된 업무보고와 질의가 이루어지지 않았다고 생각합니다. 심각한 허위 보고이거나 아니면 무능의 극치이거나 도저히 납득할 수 없는 질의응답들이 지금 이루어지고 있다는 점을 말씀드리고 이것은 민주당 우선 야당 위원님들과 의논을 해서 저는 오늘로 제대로 된 업무보고가 되지 않았기 때문에 다시 기일을 잡아서 해야 될 필요도 느낀다는 점을 말씀드리고요.  아까 존경하는 유성엽 위원님의 회의록 관련 요구에 응하지 않으셨다면 제가 위원장님께 이것은 법에 따라서 규칙을 개정하도록 요청드릴 생각이었는데 다행스럽게 위원장께서 하겠다고 말씀하신 거지요 그 점은?\"\n원자력안전위원장 강창순] \"그렇습니다.\"\n유기홍 위원] \"그것은 나중에 좀 확인해 주시기를 당부드리고요.  또 하나 말씀을 드리면 존경하는 정진후 위원께서 요청하신 자료가 한수원 관련 자료는 아마 위원회 의결을 필요로 하는 모양입니다. 이것은 위원장님께 요청드리는 것인데 사실은 한수원 자료를 제대로 받아 보지 않고 원자력안전위원회 업무보고나 이런 것들을 제대로 할 수가 없습니다.   그래서 한수원 관련된 자료에 대해서는 우리 여야 합의로 좀 자료 요청을 할 수 있도록…… 이것은 아마 다른 위원님들도 동의하실 텐데요. 한수원 자료를 제대로 받아 보지 못한다면 원자력안전위원회에 대한 보고라든지 나중에 국정감사를 또 하게 될 텐데 그 점 관련해서는 위원장께서 적극적으로 배려해 주실 것을 당부드리겠습니다.  이어서 제 질의를 드리도록 하겠습니다.   말씀드렸듯이 정말 실망스럽다는 말씀을 먼저 드리고 싶습니다.   그런데 아까 존경하는 유은혜 위원께서도 지적을 하셨지만 사실은 위원장님이 그동안 걸어오신 길을 보면 어찌 보면 당연하다는 생각도 드는 것이 사실입니다.  위원장님 작년 10월 26일 독립행정기관으로 만들어져서 사실상 제대로 된 첫 국회 업무보고일 텐데 힘들고 굉장히 곤혹스러우시지요?\"\n",
"""

In [248]:
okt = Okt()

token_nouns = okt.nouns(doc)
noun_str = ' '.join(token_nouns)
noun_str

'원자력안전위 원장 강창순 예 유성엽 위원 이상 위원장 신학용 유성엽 위원 수고 마지막 유기홍 간사 질의 유기홍 위원 제 제 질의 전 의사 진행 발언 사실 다른 위원 생각 저 오늘 제대로 업무 질의 생각 허위 무능 극치 거나 도저히 납득 수 질의 응답 지금 점 것 민주당 우선 야당 위원 의논 저 오늘 제대로 업무 보고 때문 다시 기일 필요 점 존경 유성엽 위원 회의 록 관련 요구 응 제 위원장 것 법 따라서 규칙 개정 요청 드릴 생각 다행 위원장 말씀 거지 그 점 원자력안전위 원장 강창순 유기홍 위원 그것 나중 좀 확인 시기 당부 또 하나 말씀 존경 정진후 위원 요청 자료 한수원 관련 자료 아마 위원회 의결 필요 모양 것 위원장 요청 것 사실 한수원 자료 제대로 보지 원자력안전위 회 업무 보고 것 제대로 수가 한수원 관련 자료 대해 우리 여야 합의 좀 자료 요청 수 것 아마 다른 위원 동의 한수원 자료 제대로 보지 원자력안전위 회 대한 보고 나중 국정감사 또 그 점 관련 위원장 적극 배려 것 당부 제 질의 정말 실망 말씀 먼저 존경 유은혜 위원 서도 지적 사실 위원장 그동안 길 어찌 생각 것 사실 위원장 작년 독립 행정기관 사실 제대로 첫 국회 업무 보고 일 곤혹'

In [249]:
mecab = Mecab(dicpath=r'C:\mecab\mecab-ko-dic')

In [250]:
out = mecab.pos(doc)
out

[('"', 'SY'),
 ('원자력', 'NNG'),
 ('안전', 'NNG'),
 ('위원장', 'NNG'),
 ('강창순', 'NNP'),
 (']', 'SSC'),
 ('"', 'SY'),
 ('예', 'IC'),
 ('알', 'VV'),
 ('겠', 'EP'),
 ('습니다', 'EF'),
 ('.', 'SF'),
 ('"', 'SY'),
 ('유성엽', 'NNP'),
 ('위원', 'NNG'),
 (']', 'SSC'),
 ('"', 'SY'),
 ('이상', 'NNG'),
 ('입니다', 'VCP+EF'),
 ('.', 'SF'),
 ('"', 'SY'),
 ('위원장', 'NNG'),
 ('신학용', 'NNP'),
 (']', 'SSC'),
 ('"', 'SY'),
 ('유성엽', 'NNP'),
 ('위원', 'NNG'),
 ('님', 'XSN'),
 ('수고', 'NNG'),
 ('하', 'XSV'),
 ('셨', 'EP+EP'),
 ('습니다', 'EF'),
 ('.', 'SF'),
 ('마지막', 'NNG'),
 ('으로', 'JKB'),
 ('유기홍', 'NNP'),
 ('간사', 'NNG'),
 ('님', 'XSN'),
 ('질의', 'NNG'),
 ('해', 'XSV+EC'),
 ('주', 'VX'),
 ('세요', 'EP+EF'),
 ('.', 'SF'),
 ('"', 'SY'),
 ('유기홍', 'NNP'),
 ('위원', 'NNG'),
 (']', 'SSC'),
 ('"', 'SY'),
 ('제', 'NP'),
 ('가', 'JKS'),
 ('제', 'NP+JKG'),
 ('질', 'NNG'),
 ('의', 'JKG'),
 ('전', 'NNG'),
 ('에', 'JKB'),
 ('간단', 'XR'),
 ('하', 'XSA'),
 ('게', 'EC'),
 ('의사', 'NNG'),
 ('진행', 'NNG'),
 ('발언', 'NNG'),
 ('하', 'VV'),
 ('고', 'EC'),
 ('하', 'VV'),
 ('도록', 'EC

In [251]:
noun_list = [word[0] for word in out if word[1] == 'NNG' or word[1] == 'NNP']
noun_list = [noun for noun in noun_list if len(noun) > 1]

In [252]:
stop_words = ['위원장', '위원', '선배', '저', '우리', '위', '가지', '안녕하십니까', '여러', '위해', '못', '것', '사실', '말씀', '이후', '만약', '원장', '이전', '마지막', '최소한', '생각', '사이', '그때', '해당', '결정', '장관', '오랫', '최선', '적극', '검토', '사항', '최근', '추구']
stop_names = ['강창순', '유기홍', '박성호', '유성엽', '유은혜', '정진후', '박홍근', '박윤원', '이주호', '김상희', '박혜자', '신학용']

stop_list = stop_words + stop_names

In [253]:
result = []
for w in noun_list:
    stop_bool = False
    for stop in stop_list:
        if stop in w:
            stop_bool = True
    if stop_bool != True:
        result.append(w)
                
# tokenized_nouns = ' '.join(result)

# 중복제거
# unique_token_koms_noun = set(result)
# unique_noun_list = ' '.join(unique_token_koms_noun)

# 명사 빈도 카운트
count = Counter(result)

common_list = count.most_common(10)

In [254]:
common_list

[('자료', 6),
 ('보고', 5),
 ('원자력', 4),
 ('안전', 4),
 ('업무', 4),
 ('관련', 4),
 ('요청', 4),
 ('수원', 4),
 ('질의', 3),
 ('존경', 3)]

In [255]:
tokenized_data = []
tokenized_data.append(noun_list)

In [241]:
# tokenized_data = []

# stopwords_removed_sentence = [word for word in unique_token_koms_noun if not word in stop_list] # 불용어 제거
# tokenized_data.append(stopwords_removed_sentence)

In [256]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 1, workers = 4, sg = 0)

In [243]:
#model.wv.index_to_key
#model.wv.key_to_index.keys()

In [259]:
# Word2Vec.most_similar(positive=[], negative=[], topn=10, restrict_vocab=None, indexer=None)
similar_list = model.wv.most_similar("원자력", topn=20)
similar_list

[('개정', 0.19345565140247345),
 ('안전', 0.1719825267791748),
 ('응답', 0.16499754786491394),
 ('보고', 0.14177894592285156),
 ('정진후', 0.13220402598381042),
 ('자료', 0.13207578659057617),
 ('행정', 0.11852433532476425),
 ('합의', 0.10774267464876175),
 ('의결', 0.10105088353157043),
 ('규칙', 0.10029466450214386),
 ('극치', 0.09967800229787827),
 ('당부', 0.0996745154261589),
 ('요구', 0.09951536357402802),
 ('여야', 0.09854571521282196),
 ('유은혜', 0.0851428210735321),
 ('필요', 0.07260936498641968),
 ('배려', 0.07071585208177567),
 ('말씀', 0.06914019584655762),
 ('회의록', 0.06762751936912537),
 ('요청', 0.06325370073318481)]

In [187]:
# Word2Vec.most_similar(positive=[], negative=[], topn=10, restrict_vocab=None, indexer=None)
similar_list = model.wv.most_similar("교육", topn=20)
similar_list

[('교육청', 0.2118547409772873),
 ('원칙', 0.19193443655967712),
 ('다음', 0.14378933608531952),
 ('권한', 0.1260225772857666),
 ('얼마', 0.12028773128986359),
 ('관련', 0.11691062897443771),
 ('여야', 0.11650952696800232),
 ('일선', 0.11603470891714096),
 ('개선', 0.1106594055891037),
 ('추구', 0.10782364010810852),
 ('최근', 0.10275101661682129),
 ('사업', 0.08300384134054184),
 ('적극', 0.07986127585172653),
 ('문제', 0.05614401400089264),
 ('결산', 0.046361446380615234),
 ('교과', 0.04395024850964546),
 ('국민', 0.03828371688723564),
 ('사태', 0.029528120532631874),
 ('사용', 0.027766970917582512),
 ('예산', 0.026276011019945145)]

In [260]:
sim_result = []
for sim in similar_list:
    stop_bool = False
    for stop in stop_list:
        if stop in sim[0]:
            stop_bool = True
    if stop_bool != True:
        sim_result.append((sim[0], sim[1]))

sim_result

[('개정', 0.19345565140247345),
 ('안전', 0.1719825267791748),
 ('응답', 0.16499754786491394),
 ('보고', 0.14177894592285156),
 ('자료', 0.13207578659057617),
 ('행정', 0.11852433532476425),
 ('합의', 0.10774267464876175),
 ('의결', 0.10105088353157043),
 ('규칙', 0.10029466450214386),
 ('극치', 0.09967800229787827),
 ('당부', 0.0996745154261589),
 ('요구', 0.09951536357402802),
 ('여야', 0.09854571521282196),
 ('필요', 0.07260936498641968),
 ('배려', 0.07071585208177567),
 ('회의록', 0.06762751936912537),
 ('요청', 0.06325370073318481)]